In [1]:
import os
import xmlrpc.client
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
def api_params_func(test_db: bool = False) -> dict:

    api_url = os.environ.get('ODOO_URL_API')
    api_db = os.environ.get('ODOO_DB_API')
    api_test_db = os.environ.get('ODOO_DB_PRUEBA_API')
    api_username = os.environ.get('ODOO_USERNAME_API')
    api_clave = os.environ.get('ODOO_CLAVE_API')

    api_params = {}
    api_params['api_url'] = api_url
    api_params['api_username'] = api_username
    api_params['api_clave'] = api_clave

    if test_db:
        api_params['api_db'] = api_test_db
        return api_params
    
    else:
        api_params['api_db'] = api_db
        return api_params

In [3]:
def if_list_gt0_idex (item: dict, key: str) -> None | int:
        val = item[key]

        if val:
            if len(val) == 0:
                return None
            else:
                return val[0]
        else:
            return None

In [4]:
def search_fact_func(mes: int, dias: None | list[int] = None, vendedor: None | int  = None) -> list[str]:
    
    if type(mes) != int or mes < 1 or mes > 12:
        raise Exception (f'El mes es incorrecto. El párametro "mes" debe ser un número entero entre 1 y 12. Escribiste: {mes}')
    

    if dias is None:
        param_dia_ini = datetime(2024, mes, 1)
        param_dia_fin = datetime(2024, mes + 1, 1) - timedelta(days= 1)
    
    elif len(dias) != 2:
        raise Exception (f'El párametro "días_del_mes" debe ser una lista de 2 elementos. El día inicial de búsqueda se escribe en el índice 0 de la lista, el día final de búsqueda en el índice 1. La lista tiene: {len(dias)} de elementos')
    
    elif type(dias[0]) != int or type(dias[1]) != int:
        raise Exception (f'El párametro "día_inicial_de_búsqueda" y "día_final_de_búsqueda" sólo pueden ser números enteros.')

    elif dias[0] > dias[1]:
        raise Exception (f'El párametro "día_inicial_de_búsqueda" no debe ser mayor al parámetro "día_final_de_búsqueda". El día inicial de búsqueda se escribe en el índice 0 de la lista, el día final de búsqueda en el índice 1.')
    
    else:
        try:
            param_dia_ini = datetime(2024, mes, dias[0])
        except:
            raise Exception (f'El párametro "día_inicial_de_búsqueda" es incorrecto. La fecha "día: {dias[0]}, mes: {mes}, año 2024" no existe')
        try:
            param_dia_fin = datetime(2024, mes, dias[1])
        except:
            raise Exception (f'El párametro "día_final_de_búsqueda" es incorrecto. La fecha "día: {dias[1]}, mes: {mes}, año 2024" no existe')


    search_fact_all = [
        "&",
            ("state", "=", "posted"),
        "&", "&",
            ("invoice_date", ">=", param_dia_ini.strftime('%Y-%m-%d')),
            ("invoice_date", "<=", param_dia_fin.strftime('%Y-%m-%d')),
            ("journal_id", "in", [10, 90, 30, 97])
        ]

    search_fact_vendedor = [
        "&",
            ("state", "=", "posted"),
        "&", "&",
            ("invoice_date", ">=", param_dia_ini.strftime('%Y-%m-%d')),
            ("invoice_date", "<=", param_dia_fin.strftime('%Y-%m-%d')),
        "&",
            ("journal_id", "in", [10, 90, 30, 97]),
        "|",
            ("invoice_user_id", "=", vendedor),
            ("pos_order_ids.lines.sale_order_origin_id.user_id", "=", vendedor)]


    if vendedor == None:
        return search_fact_all
    
    if type(vendedor) != int:
        raise Exception (f'El párametro "Vendedor" debe ser un número entero. Escribiste: {vendedor}')
    
    return search_fact_vendedor

In [5]:
def api_call_func(api_params: dict, search_fact: list ) -> list[int | dict]:
    
    api_url = api_params['api_url']
    api_db = api_params['api_db']
    api_username = api_params['api_username']
    api_clave = api_params['api_clave']

    common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
    uid = common.authenticate(api_db, api_username, api_clave, {})
    models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')


    search_fact = search_fact


    fact_doc_fields = [
                    'name',
                    'invoice_date',
                    'state',
                    'reversed_entry_id',
                    'reversal_move_id',
                    'journal_id',
                    'company_id',
                    'invoice_origin',
                    'pos_order_ids',
                    'line_ids',
                    'partner_id',
                    'move_type',
                    'invoice_user_id'
                    ]

    fact_doc_ids1 = models.execute_kw(api_db, uid, api_clave, 'account.move', 'search', [search_fact])
    fact_doc_ids2 = models.execute_kw(api_db, uid, api_clave, 'account.move', 'search', [[("reversal_move_id", "=", fact_doc_ids1)]])
    fact_doc_ids = list(set(fact_doc_ids1) | set(fact_doc_ids2))
    fact_doc_ids.sort()

    fact_doc_json = models.execute_kw(api_db, uid, api_clave, 'account.move', 'read', [fact_doc_ids], {'fields': fact_doc_fields})

    fact_line_ids = []
    pos_doc_ids1 = []
    invoice_origin_names = []

    for fact in fact_doc_json:
        if fact['pos_order_ids']:
            pos_doc_ids1.append(fact['pos_order_ids'][0])
        
        if fact['invoice_origin']:
            if fact['invoice_origin'][:2] in ['Pd', 'Sh']:
                invoice_origin_names.append(fact['invoice_origin'])

        fact_line_ids += fact['line_ids']


    fact_line_fields = [
            'product_id',
            'quantity',
            'price_unit',
            'discount',
            'account_id',
            'price_subtotal',
        ]


    fact_line_ids.sort()
    fact_line_json = models.execute_kw(api_db, uid, api_clave, 'account.move.line', 'read', [fact_line_ids], {'fields': fact_line_fields})


    pos_line_fields = [
        'name',
        'order_id',
        'sale_order_origin_id',
        'refunded_orderline_id',
    ]

    pos_doc_ids2 = models.execute_kw(api_db, uid, api_clave, 'pos.order', 'search', [[("name", "in", invoice_origin_names)]])
    pos_doc_ids = list(set(pos_doc_ids1) | set(pos_doc_ids2))
    pos_doc_ids.sort()

    pos_line_ids1 = models.execute_kw(api_db, uid, api_clave, 'pos.order.line', 'search', [[("order_id.id", "=", pos_doc_ids)]])
    pos_line_ids2 = models.execute_kw(api_db, uid, api_clave, 'pos.order.line', 'search', [[("refund_orderline_ids", "=", pos_line_ids1)]])
    pos_line_ids = list(set(pos_line_ids1) | set(pos_line_ids2))
    pos_line_ids.sort()
    pos_line_json = models.execute_kw(api_db, uid, api_clave, 'pos.order.line', 'read', [pos_line_ids], {'fields': pos_line_fields})


    sale_doc_ids_each_line = []
    for pos in pos_line_json:
        if pos['sale_order_origin_id']:
            sale_doc_ids_each_line.append(pos['sale_order_origin_id'][0])

    sale_doc_ids = list(set(sale_doc_ids_each_line))
    sale_doc_ids.sort()


    sale_doc_fields = [
        'user_id',
    ]

    sale_doc_json = models.execute_kw(api_db, uid, api_clave, 'sale.order', 'read', [sale_doc_ids], {'fields': sale_doc_fields})

    return fact_doc_ids1, fact_doc_json, fact_line_json, pos_line_json, sale_doc_json

In [6]:
def fact_doc_df_fun(fact_doc_ids: list[int], fact_doc_json: list[dict]) -> pd.DataFrame:
    
    data_fact = []

    for fact in fact_doc_json:
        if fact['id'] in fact_doc_ids:
            for line in fact['line_ids']:
                new = {}
                new['fact_doc_id'] = fact['id']
                new['name'] = fact['name']
                new['invoice_date'] = fact['invoice_date']
                new['state'] = fact['state']
                new['invoice_origin'] = fact['invoice_origin']
                new['module_origin'] = None
                new['pos_doc_id'] = if_list_gt0_idex(fact, 'pos_order_ids')
                new['move_type'] = fact['move_type']
                new['reversal_move_id'] = if_list_gt0_idex(fact, 'reversal_move_id')
                new['reversed_entry_id'] = if_list_gt0_idex(fact, 'reversed_entry_id')
                new['journal_id'] = fact['journal_id'][0]
                new['company_id'] = fact['company_id'][0]
                new['partner_id'] = fact['partner_id'][0]
                new['invoice_user_id'] = fact['invoice_user_id'][0]
                new['fact_line_id'] = line

                if not fact['invoice_origin']:
                    new['module_origin'] = 'Contabilidad'
                elif fact['invoice_origin'][:2] in ['Pd', 'Sh']:
                        new['module_origin'] = 'PdV'
                elif fact['invoice_origin'][0] == 'S':
                        new['module_origin'] = 'Ventas'

                data_fact.append(new)



    fact_doc_df = pd.DataFrame(data_fact)


    fact_doc_df.loc[fact_doc_df['invoice_origin'] == False , ['invoice_origin',]] = pd.NA
    fact_doc_df['invoice_date'] = pd.to_datetime(fact_doc_df['invoice_date'], format='%Y-%m-%d')
    
    for col in fact_doc_df.columns:
         if fact_doc_df[col].dtype in ['int64', 'float64']:
              fact_doc_df[col] = fact_doc_df[col].astype('Int64')

    return fact_doc_df

In [7]:
def fact_line_df_fun(fact_line_json: list[dict]) -> pd.DataFrame:

    data_line_fact = []

    for fact_line in fact_line_json:
        if fact_line['account_id'] and fact_line['account_id'][0] in [85, 197]:
            new = {}
            new['fact_line_id'] = fact_line['id']
            new['product_id'] = fact_line['product_id'][0]
            new['quantity'] = fact_line['quantity']
            new['price_unit'] = fact_line['price_unit']
            new['discount'] = fact_line['discount'] / 100
            new['price_subtotal'] = fact_line['price_subtotal']

            data_line_fact.append(new)


    fact_line_df = pd.DataFrame(data_line_fact)


    fact_line_df.loc[fact_line_df['product_id'] == False, ['product_id',]] = pd.NA
    fact_line_df['fact_line_id'] = fact_line_df['fact_line_id'].astype('Int64')
    fact_line_df['product_id'] = fact_line_df['product_id'].astype('Int64')
    
    return fact_line_df

In [8]:
def pos_line_df_fun(pos_line_json: list[dict]) -> pd.DataFrame:

    data_pos_line = []

    for pos in pos_line_json:
        new = {}
        new['pos_line_id'] = pos['id']
        new['pos_doc_id'] = pos['order_id'][0]
        new['sale_doc_id'] = if_list_gt0_idex(pos, 'sale_order_origin_id')
        new['refunded_orderline_id'] = if_list_gt0_idex(pos, 'refunded_orderline_id')
        

        data_pos_line.append(new)

    pos_line_df = pd.DataFrame(data_pos_line)
    
    for col in pos_line_df.columns:
         if pos_line_df[col].dtype in ['int64', 'float64']:
              pos_line_df[col] = pos_line_df[col].astype('Int64')

    
    # Para copiar el 'sale_doc_id' de la línea origial de venta a la línea de devolución que no tiene.
    for id in pos_line_df.loc[(pos_line_df['sale_doc_id'].isna()) & (~pos_line_df['refunded_orderline_id'].isna())]['refunded_orderline_id'].unique():
        pos_line_df.loc[(pos_line_df['refunded_orderline_id'] == id), 'sale_doc_id'] = pos_line_df.loc[(pos_line_df['pos_line_id'] == id), 'sale_doc_id'].iloc[0]

    # Para copiar el 'sale_doc_id' de una línea hermana con el mismo pos_doc_id.
    pos_line_id_not_sale_doc, pos_doc_id_not_sale_doc = pos_line_df.loc[pos_line_df['sale_doc_id'].isna(), ['pos_line_id', 'pos_doc_id']].items()
    for i in range(len(pos_line_id_not_sale_doc[1])):
        pos_line_df.loc[(pos_line_df['pos_line_id'] == pos_line_id_not_sale_doc[1].iloc[i]), 'sale_doc_id'] = pos_line_df.loc[(pos_line_df['pos_doc_id'] == pos_doc_id_not_sale_doc[1].iloc[i]), 'sale_doc_id'].iloc[0]


    return pos_line_df

In [9]:
def sale_doc_df_fun(sale_doc_json: list[dict]) -> pd.DataFrame:

    data_sale_doc = []

    for sale in sale_doc_json:
        new = {}
        new['sale_doc_id'] = sale['id']
        new['salesperson_id'] = sale['user_id'][0]

        data_sale_doc.append(new)


    sale_doc_df = pd.DataFrame(data_sale_doc)
    for col in sale_doc_df.columns:
        sale_doc_df[col] = sale_doc_df[col].astype('Int64')

    return sale_doc_df

In [10]:
def fact_df_func(fact_doc_df: pd.DataFrame, fact_line_df: pd.DataFrame) -> pd.DataFrame:

    fact_df = fact_doc_df.merge(fact_line_df, how='left', on='fact_line_id')
    fact_df.loc[fact_df['move_type'] == 'out_refund', ['quantity', 'price_subtotal']] = fact_df.loc[fact_df['move_type'] == 'out_refund', ['quantity', 'price_subtotal']] * -1
    fact_df.loc[fact_df['module_origin'] != 'PdV', 'salesperson_id'] = fact_df.loc[fact_df['module_origin'] != 'PdV', 'invoice_user_id']

    return fact_df

In [11]:
api_params = api_params_func()
search_fact = search_fact_func(3)

In [20]:
fact_doc_ids, fact_doc_json, fact_line_json, pos_line_json, sale_doc_json = api_call_func(api_params, search_fact)

In [13]:
fact_doc_df = fact_doc_df_fun(fact_doc_ids, fact_doc_json)
fact_line_df = fact_line_df_fun(fact_line_json)
pos_line_df = pos_line_df_fun(pos_line_json)
sale_doc_df = sale_doc_df_fun(sale_doc_json)

fact_df = fact_df_func(fact_doc_df, fact_line_df)

In [14]:
pos_sale_df = pos_line_df.merge(sale_doc_df, how='left', on='sale_doc_id')

In [15]:
pos_sale_df[~pos_sale_df['salesperson_id'].isna()]

,pos_line_id,pos_doc_id,sale_doc_id,refunded_orderline_id,salesperson_id
0,11852,4735,6765,<NA>,234
1,11853,4735,6765,<NA>,234
2,11854,4735,6765,<NA>,234
3,11855,4735,6765,<NA>,234
4,11856,4735,6765,<NA>,234
...,...,...,...,...,...
8868,28108,11223,15390,<NA>,212
8869,28109,11224,15501,<NA>,214
8870,28110,11225,15504,<NA>,214
8871,28111,11225,15504,<NA>,214


In [16]:
pos_sale_df.loc[pos_sale_df['pos_doc_id'] == 4735, ['pos_doc_id', 'salesperson_id']].groupby('pos_doc_id').first().iloc[0, 0]

234

In [17]:
pos_sale_df.loc[pos_sale_df['pos_doc_id'] == 4735, ['pos_doc_id', 'salesperson_id']].groupby('pos_doc_id').mean().iloc[0,0]

234.0

In [18]:
wep = pos_sale_df.loc[:, ['pos_doc_id', 'salesperson_id']].groupby('pos_doc_id').agg({'salesperson_id':['first', 'mean']})
wep['wep'] = wep[('salesperson_id', 'first')] == wep[('salesperson_id', 'mean')]
wep.loc[wep['wep'] == False]
wep.loc[wep['wep'] == True]
wep2 = wep.loc[wep['wep'] == True, ('salesperson_id', 'first')]
wep2.name = 'wep'
wep2

pos_doc_id
4735     234
6726     224
6833     224
7555     212
7581     212
        ... 
11221    214
11222    212
11223    212
11224    214
11225    214
Name: wep, Length: 3486, dtype: Int64

In [19]:
fact_df.merge(wep2, how='left', on='pos_doc_id').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18457 entries, 0 to 18456
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   fact_doc_id        18457 non-null  Int64         
 1   name               18457 non-null  object        
 2   invoice_date       18457 non-null  datetime64[ns]
 3   state              18457 non-null  object        
 4   invoice_origin     18454 non-null  object        
 5   module_origin      18457 non-null  object        
 6   pos_doc_id         15803 non-null  Int64         
 7   move_type          18457 non-null  object        
 8   reversal_move_id   133 non-null    Int64         
 9   reversed_entry_id  137 non-null    Int64         
 10  journal_id         18457 non-null  Int64         
 11  company_id         18457 non-null  Int64         
 12  partner_id         18457 non-null  Int64         
 13  invoice_user_id    18457 non-null  Int64         
 14  fact_l